In [ ]:
import random
random.seed(42)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt

import torch
from torch import optim, nn
from torch.utils.data import DataLoader, TensorDataset, Dataset, random_split
from torchvision.utils import make_grid
from torchvision import transforms as T
from torchvision import models, datasets
from random import randint

from tqdm import tqdm
import os

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

In [ ]:

train_data = datasets.FashionMNIST('./', train=True, download=True)

# Stick all the images together to form a 600000 X 28 array
x = np.concatenate([np.asarray(train_data[i][0]) for i in range(len(train_data))])

# calculate the mean and std along the (0, 1) axes
mean = np.mean(x, axis=(0, 1))/255
std = np.std(x, axis=(0, 1))/255
# the the mean and std
mean=mean.tolist()
std=std.tolist()

In [ ]:
print(f'mean = {mean}, std = {std}')

In [ ]:
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()
    
def show_batch(dataloader):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)    
    imshow(make_grid(images)) # Using Torchvision.utils make_grid function
    
def show_image(dataloader):
    dataiter = iter(dataloader)
    images, labels = next(dataiter)
    random_num = randint(0, len(images)-1)
    imshow(images[random_num])
    label = labels[random_num]
    print(f'Label: {label}, Shape: {images[random_num].shape}')

In [ ]:
# Transformation - optional depending on future resnet implementation

# Define transformation sequence for image pre-processing
# If not using pre-trained model, normalize with 0.5, 0.5, 0.5 (mean and SD)
# If using pre-trained ImageNet, normalize with mean=[0.485, 0.456, 0.406], 
# std=[0.229, 0.224, 0.225])

train_transform = T.Compose([
    T.Resize((32,32)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(15),
    T.ToTensor(),
    T.Normalize(mean, std)
])

test_transform = T.Compose([
                # T.Resize(256), # Resize images to 256 x 256
                # T.CenterCrop(224), # Center crop image
                # T.RandomHorizontalFlip(),
                T.Resize((32,32)),
                T.ToTensor(),  # Converting cropped images to tensors
                T.Normalize(mean, std)
])


In [ ]:
batch_size = 128

In [ ]:
trainset = datasets.FashionMNIST("./",
                                         train=True,
                                         download=True,
                                         transform=train_transform)
train_loader = torch.utils.data.DataLoader(
    trainset, batch_size, shuffle=True, num_workers=2,pin_memory=True)

testset = datasets.FashionMNIST("./",
                                        train=False,
                                        download=True,
                                        transform=test_transform)
test_loader = torch.utils.data.DataLoader(
    testset, batch_size*2,pin_memory=True, num_workers=2)


In [ ]:
show_image(train_loader)


In [ ]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
def test():
    net = LeNet5(10)
    y = net(torch.randn(1, 1, 32, 32))
    print(y.size())
test()

In [ ]:
len(test_loader.dataset)

In [ ]:
dataloaders = {
    'train': train_loader,
    'val': test_loader,
    'test': test_loader
}
dataset_sizes = {
    'train': len(train_loader.dataset),
    'val': len(test_loader.dataset),
    'test': len(test_loader.dataset),
}

In [ ]:
def loss_fn_kd(outputs, labels, teacher_outputs, **kwargs):
    """
    Compute the knowledge-distillation (KD) loss given outputs, labels.
    "Hyperparameters": temperature and alpha
    NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
    and student expects the input tensor to be log probabilities! See Issue #2
    """
    teacher_outputs = teacher_outputs.double()
    alpha = kwargs['alpha']
    T = kwargs['temperature']

    loss_CE = F.cross_entropy(outputs, labels)
    D_KL = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1), teacher_outputs) * (T * T)
    KD_loss =  (1. - alpha)*loss_CE + alpha*D_KL

    return KD_loss

In [ ]:
soft_probabilities = torch.tensor(torch.load('./tiny_ae_fashion_mnist.pt'))
soft_probabilities = F.softmax(torch.tensor(soft_probabilities), dim = 1)
print(soft_probabilities.shape)
print(soft_probabilities.type())
soft_probabilities = soft_probabilities.to(device)
print(soft_probabilities[0])

In [ ]:
print(soft_probabilities.device)

In [ ]:
loss_fn_kd(
    torch.randn(128,200),
    torch.randn(128,200),
    F.softmax(torch.randn(128,200), dim=1),
    alpha = 0.65,
    temperature = 1
    # {'alpha': 0.65, 'temperature': 1}
)

In [ ]:
import time
import copy

def train_model(model, criterion_kd, optimizer, scheduler, alpha, num_epochs=25, temp=1):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(1, num_epochs+1):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                teacher_outputs = soft_probabilities[labels]

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = outputs.double()
                    # print(outputs.type())
                    _, preds = torch.max(outputs, 1)
                    loss = criterion_kd(outputs, labels, teacher_outputs, alpha=alpha, temperature=temp)
                    # print(loss.type())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                print('Saving..')
                state = {
                    'model': model.state_dict(),
                    'acc': epoch_acc,
                    'epoch': epoch,
                }
                # if not os.path.isdir('checkpoint'):
                #     os.mkdir('checkpoint')
                torch.save(state, './lenet_distilled_tiny_ae.pth')
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
import time
import copy

def accuracyAtk(output, target, topk=(1,5)):
    """
    Computes the accuracy over the k top predictions for the specified values of k
    In top-5 accuracy you give yourself credit for having the right answer
    if the right answer appears in your top five guesses.
    """
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = (pred == target.unsqueeze(dim=0)).expand_as(pred)

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k)
        return res

def eval_model(model):
    since = time.time()

    model.eval()

    final_top_1, final_top_5 = 0,0

    for inputs, labels in tqdm(dataloaders['test']):
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            top1, top5 = accuracyAtk(outputs, labels)
            final_top_1 += top1
            final_top_5 += top5
    
    top1_acc = final_top_1/dataset_sizes['test']
    top5_acc = final_top_5/dataset_sizes['test']
    print(f'Top 1 test accuracy = {top1_acc}, Top 5 test accuract = {top5_acc}')

In [ ]:
from torch.optim import lr_scheduler
for t in [20, 10, 5, 1]:
    for i in range(1, 10, 1):
        model_ft = LeNet5(num_classes=10)
        model_ft = model_ft.to(device)

        criterion = loss_fn_kd
        optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
        exp_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer_ft, milestones=[60, 120, 160], gamma=0.2)
        model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, i/10, num_epochs=200, temp=t)
        eval_model(model_ft)
        del model_ft